In [1]:
# Libraries

import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./t10k-labels-idx1-ubyte.gz


In [2]:
# Initialisation

# Parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

# Number of samples to calculate validation and accuracy
# Decrease this if you're running out of memory to calculate accuracy
test_valid_size = 256

# Network Parameters
n_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.75  # Dropout, probability to keep units


# Store layers weight & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}


In [3]:
# Functions to define CNN

def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    # Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out


In [4]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.})

            print('Epoch {:>2}, Batch {:>3} - Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))

Epoch  1, Batch   1 - Loss: 73930.5312 Validation Accuracy: 0.136719
Epoch  1, Batch   2 - Loss: 55880.7812 Validation Accuracy: 0.140625
Epoch  1, Batch   3 - Loss: 47814.1445 Validation Accuracy: 0.125000
Epoch  1, Batch   4 - Loss: 53781.5156 Validation Accuracy: 0.148438
Epoch  1, Batch   5 - Loss: 41751.3125 Validation Accuracy: 0.148438
Epoch  1, Batch   6 - Loss: 34910.1758 Validation Accuracy: 0.140625
Epoch  1, Batch   7 - Loss: 35641.4883 Validation Accuracy: 0.144531
Epoch  1, Batch   8 - Loss: 34804.4570 Validation Accuracy: 0.148438
Epoch  1, Batch   9 - Loss: 31675.7832 Validation Accuracy: 0.144531
Epoch  1, Batch  10 - Loss: 28768.5176 Validation Accuracy: 0.152344
Epoch  1, Batch  11 - Loss: 30468.7109 Validation Accuracy: 0.156250
Epoch  1, Batch  12 - Loss: 25852.8047 Validation Accuracy: 0.175781
Epoch  1, Batch  13 - Loss: 21144.0703 Validation Accuracy: 0.179688
Epoch  1, Batch  14 - Loss: 21427.5566 Validation Accuracy: 0.199219
Epoch  1, Batch  15 - Loss: 18250.

Epoch  1, Batch 122 - Loss:  2780.8557 Validation Accuracy: 0.558594
Epoch  1, Batch 123 - Loss:  3659.8660 Validation Accuracy: 0.562500
Epoch  1, Batch 124 - Loss:  3862.9817 Validation Accuracy: 0.566406
Epoch  1, Batch 125 - Loss:  4618.4907 Validation Accuracy: 0.570312
Epoch  1, Batch 126 - Loss:  3575.2942 Validation Accuracy: 0.585938
Epoch  1, Batch 127 - Loss:  2564.9563 Validation Accuracy: 0.601562
Epoch  1, Batch 128 - Loss:  4103.9277 Validation Accuracy: 0.609375
Epoch  1, Batch 129 - Loss:  3959.9143 Validation Accuracy: 0.597656
Epoch  1, Batch 130 - Loss:  3492.0293 Validation Accuracy: 0.609375
Epoch  1, Batch 131 - Loss:  3648.0171 Validation Accuracy: 0.597656
Epoch  1, Batch 132 - Loss:  3486.9377 Validation Accuracy: 0.597656
Epoch  1, Batch 133 - Loss:  2881.6807 Validation Accuracy: 0.593750
Epoch  1, Batch 134 - Loss:  4609.7681 Validation Accuracy: 0.589844
Epoch  1, Batch 135 - Loss:  4361.1772 Validation Accuracy: 0.597656
Epoch  1, Batch 136 - Loss:  3861.

Epoch  1, Batch 261 - Loss:  2532.4712 Validation Accuracy: 0.671875
Epoch  1, Batch 262 - Loss:  1676.5986 Validation Accuracy: 0.660156
Epoch  1, Batch 263 - Loss:  2401.0283 Validation Accuracy: 0.679688
Epoch  1, Batch 264 - Loss:  1489.9475 Validation Accuracy: 0.679688
Epoch  1, Batch 265 - Loss:  1760.4120 Validation Accuracy: 0.664062
Epoch  1, Batch 266 - Loss:  2275.7864 Validation Accuracy: 0.667969
Epoch  1, Batch 267 - Loss:  1189.0454 Validation Accuracy: 0.671875
Epoch  1, Batch 268 - Loss:  1841.1760 Validation Accuracy: 0.664062
Epoch  1, Batch 269 - Loss:  2785.6216 Validation Accuracy: 0.667969
Epoch  1, Batch 270 - Loss:  2109.2261 Validation Accuracy: 0.660156
Epoch  1, Batch 271 - Loss:  1767.7150 Validation Accuracy: 0.656250
Epoch  1, Batch 272 - Loss:  2870.8953 Validation Accuracy: 0.656250
Epoch  1, Batch 273 - Loss:  2178.2976 Validation Accuracy: 0.667969
Epoch  1, Batch 274 - Loss:  1989.1780 Validation Accuracy: 0.667969
Epoch  1, Batch 275 - Loss:  1969.

Epoch  1, Batch 389 - Loss:  1478.2136 Validation Accuracy: 0.699219
Epoch  1, Batch 390 - Loss:  1993.8572 Validation Accuracy: 0.699219
Epoch  1, Batch 391 - Loss:  1738.0419 Validation Accuracy: 0.699219
Epoch  1, Batch 392 - Loss:  1002.7816 Validation Accuracy: 0.707031
Epoch  1, Batch 393 - Loss:  1117.3940 Validation Accuracy: 0.703125
Epoch  1, Batch 394 - Loss:  1033.1655 Validation Accuracy: 0.703125
Epoch  1, Batch 395 - Loss:  1486.8750 Validation Accuracy: 0.699219
Epoch  1, Batch 396 - Loss:  1827.4541 Validation Accuracy: 0.703125
Epoch  1, Batch 397 - Loss:  1094.2961 Validation Accuracy: 0.695312
Epoch  1, Batch 398 - Loss:  1550.3525 Validation Accuracy: 0.695312
Epoch  1, Batch 399 - Loss:   746.4884 Validation Accuracy: 0.695312
Epoch  1, Batch 400 - Loss:  1256.1757 Validation Accuracy: 0.703125
Epoch  1, Batch 401 - Loss:  1812.4241 Validation Accuracy: 0.699219
Epoch  1, Batch 402 - Loss:   889.5980 Validation Accuracy: 0.699219
Epoch  1, Batch 403 - Loss:  1419.

Epoch  2, Batch  84 - Loss:  1366.7134 Validation Accuracy: 0.738281
Epoch  2, Batch  85 - Loss:  1139.0680 Validation Accuracy: 0.742188
Epoch  2, Batch  86 - Loss:  1280.8777 Validation Accuracy: 0.750000
Epoch  2, Batch  87 - Loss:  1095.5630 Validation Accuracy: 0.746094
Epoch  2, Batch  88 - Loss:  1217.5511 Validation Accuracy: 0.753906
Epoch  2, Batch  89 - Loss:  1306.1305 Validation Accuracy: 0.742188
Epoch  2, Batch  90 - Loss:  1351.4777 Validation Accuracy: 0.742188
Epoch  2, Batch  91 - Loss:  1000.4797 Validation Accuracy: 0.738281
Epoch  2, Batch  92 - Loss:  1306.3774 Validation Accuracy: 0.734375
Epoch  2, Batch  93 - Loss:  1399.4504 Validation Accuracy: 0.730469
Epoch  2, Batch  94 - Loss:  1025.1805 Validation Accuracy: 0.726562
Epoch  2, Batch  95 - Loss:  1161.8396 Validation Accuracy: 0.726562
Epoch  2, Batch  96 - Loss:  1158.7828 Validation Accuracy: 0.730469
Epoch  2, Batch  97 - Loss:   987.9509 Validation Accuracy: 0.734375
Epoch  2, Batch  98 - Loss:  1304.

...

Epoch  9, Batch 361 - Loss:   249.7822 Validation Accuracy: 0.855469
Epoch  9, Batch 362 - Loss:   388.6262 Validation Accuracy: 0.859375
Epoch  9, Batch 363 - Loss:   249.9041 Validation Accuracy: 0.859375
Epoch  9, Batch 364 - Loss:   165.9140 Validation Accuracy: 0.855469
Epoch  9, Batch 365 - Loss:   258.6397 Validation Accuracy: 0.851562
Epoch  9, Batch 366 - Loss:   236.1828 Validation Accuracy: 0.851562
Epoch  9, Batch 367 - Loss:   283.9636 Validation Accuracy: 0.851562
Epoch  9, Batch 368 - Loss:   406.0987 Validation Accuracy: 0.843750
Epoch  9, Batch 369 - Loss:   238.9852 Validation Accuracy: 0.851562
Epoch  9, Batch 370 - Loss:   198.7954 Validation Accuracy: 0.851562
Epoch  9, Batch 371 - Loss:   141.5807 Validation Accuracy: 0.851562
Epoch  9, Batch 372 - Loss:   276.7968 Validation Accuracy: 0.847656
Epoch  9, Batch 373 - Loss:   269.0562 Validation Accuracy: 0.851562
Epoch  9, Batch 374 - Loss:   241.3150 Validation Accuracy: 0.855469
Epoch  9, Batch 375 - Loss:   370.

Epoch 10, Batch  54 - Loss:   421.8093 Validation Accuracy: 0.863281
Epoch 10, Batch  55 - Loss:   179.9451 Validation Accuracy: 0.863281
Epoch 10, Batch  56 - Loss:   363.3858 Validation Accuracy: 0.867188
Epoch 10, Batch  57 - Loss:   295.1839 Validation Accuracy: 0.863281
Epoch 10, Batch  58 - Loss:   300.6618 Validation Accuracy: 0.867188
Epoch 10, Batch  59 - Loss:   180.5587 Validation Accuracy: 0.867188
Epoch 10, Batch  60 - Loss:   301.6985 Validation Accuracy: 0.867188
Epoch 10, Batch  61 - Loss:   355.0816 Validation Accuracy: 0.863281
Epoch 10, Batch  62 - Loss:   194.9886 Validation Accuracy: 0.867188
Epoch 10, Batch  63 - Loss:   203.4608 Validation Accuracy: 0.871094
Epoch 10, Batch  64 - Loss:   177.0164 Validation Accuracy: 0.867188
Epoch 10, Batch  65 - Loss:   253.7520 Validation Accuracy: 0.867188
Epoch 10, Batch  66 - Loss:   254.4864 Validation Accuracy: 0.867188
Epoch 10, Batch  67 - Loss:   124.2943 Validation Accuracy: 0.867188
Epoch 10, Batch  68 - Loss:   295.

Epoch 10, Batch 182 - Loss:   243.8191 Validation Accuracy: 0.847656
Epoch 10, Batch 183 - Loss:   150.2934 Validation Accuracy: 0.851562
Epoch 10, Batch 184 - Loss:   355.1043 Validation Accuracy: 0.851562
Epoch 10, Batch 185 - Loss:   251.8223 Validation Accuracy: 0.851562
Epoch 10, Batch 186 - Loss:    98.2810 Validation Accuracy: 0.851562
Epoch 10, Batch 187 - Loss:   276.0340 Validation Accuracy: 0.851562
Epoch 10, Batch 188 - Loss:   202.1446 Validation Accuracy: 0.851562
Epoch 10, Batch 189 - Loss:   274.9359 Validation Accuracy: 0.847656
Epoch 10, Batch 190 - Loss:   271.4583 Validation Accuracy: 0.851562
Epoch 10, Batch 191 - Loss:   297.0410 Validation Accuracy: 0.855469
Epoch 10, Batch 192 - Loss:   343.1178 Validation Accuracy: 0.855469
Epoch 10, Batch 193 - Loss:   271.9200 Validation Accuracy: 0.855469
Epoch 10, Batch 194 - Loss:   353.9160 Validation Accuracy: 0.851562
Epoch 10, Batch 195 - Loss:   227.7708 Validation Accuracy: 0.847656
Epoch 10, Batch 196 - Loss:   357.

Epoch 10, Batch 307 - Loss:   201.5012 Validation Accuracy: 0.855469
Epoch 10, Batch 308 - Loss:   298.0888 Validation Accuracy: 0.855469
Epoch 10, Batch 309 - Loss:   235.7083 Validation Accuracy: 0.851562
Epoch 10, Batch 310 - Loss:   195.1136 Validation Accuracy: 0.855469
Epoch 10, Batch 311 - Loss:   274.9692 Validation Accuracy: 0.859375
Epoch 10, Batch 312 - Loss:   222.8488 Validation Accuracy: 0.859375
Epoch 10, Batch 313 - Loss:   122.5709 Validation Accuracy: 0.855469
Epoch 10, Batch 314 - Loss:   276.8818 Validation Accuracy: 0.855469
Epoch 10, Batch 315 - Loss:   255.1952 Validation Accuracy: 0.859375
Epoch 10, Batch 316 - Loss:   157.3150 Validation Accuracy: 0.859375
Epoch 10, Batch 317 - Loss:   242.5860 Validation Accuracy: 0.863281
Epoch 10, Batch 318 - Loss:   263.8222 Validation Accuracy: 0.859375
Epoch 10, Batch 319 - Loss:   269.4513 Validation Accuracy: 0.851562
Epoch 10, Batch 320 - Loss:   258.6169 Validation Accuracy: 0.859375
Epoch 10, Batch 321 - Loss:   212.

Testing Accuracy: 0.83984375
